# (Unofficial) Baseball Stats API

## Goal: Excel spreadsheet that has last 10 sesasons stats for all players
- If you want to only look at last years stats, you can filter by '2021'
- If you want to look at a players career stats, you can filter by player/player_id

In [ ]:
from baseball import api


api.PlayerData().search_for_players("Jose Ramirez")

In [ ]:
player_data

In [ ]:
player_details = Baseball().player_data.get_player_details("493316")
player_details

In [ ]:
search_results = stats.search_player_all("Trea Turner")

player_id = search_results[0].get("player_id")
player_info = stats.player_info(player_id)

player_info

In [ ]:
import requests
from urllib.parse import urlencode, quote_plus
from baseball import Baseball


player_data = Baseball().player_data.search_for_players("Aaron Judge")
player_id = player_data[0]["player_id"]

params = {}
params["player_id"] = player_id
params["game_type"] = f"'R'"
params["season"] = "2015"
params["league_list_id"] = "'mlb'"

encoded_params = urlencode(params, quote_via=quote_plus)

url = f"http://lookup-service-prod.mlb.com/json/named.sport_hitting_tm.bam?{encoded_params}"
r = requests.get(url)
r.json()["sport_hitting_tm"]["queryResults"].get("row")

### Get year by year stats by player name

In [ ]:
from baseball import Baseball
import pandas as pd


player_data = Baseball().player_data.search_for_players("Christian Yelich")
player_id = player_data[0]["player_id"]


seasons = ["2021", "2020", "2019", "2018", "2017", "2016"]

player_stats = []
for year in seasons:
    hitting_stats = Baseball().stats_data.season_hitting_stats(player_id, season=year)
    if hitting_stats is None:
        break
    else:
        player_stats.append(hitting_stats)

df = pd.DataFrame(player_stats)
df


### Get Teams by Season

In [ ]:
from baseball import api


season_teams = api.TeamData().get_teams_by_season("2022")
season_teams


In [ ]:
api.TeamData().get_teams_by_season("2022")

In [ ]:
import pandas as pd


df = pd.DataFrame(season_teams)
df

### Get 40 man roster

In [ ]:
import pandas as pd


df = pd.DataFrame(mlb_roster)
df.to_excel("mlb_roster.xlsx")

In [ ]:
from baseball import api
from tqdm import tqdm, trange
from datetime import datetime, date
import time


def calculate_age(birth_date, season_year):
    today = date(year=int(season_year), month=4, day=1)
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age



# team_list = api.TeamData().get_teams_by_season("2022")

# mlb_roster = []
# for team in trange(len(team_list)):
#     team_id = team_list[team].get("team_id")
#     team_roster = api.TeamData().get_40_man_roster(team_id)
#     mlb_roster.extend(team_roster)
#     time.sleep(.1)

# print(f"Found {len(mlb_roster)} total players.")

seasons = ["2022", "2021", "2020", "2019", "2018", "2017", "2016"]

all_stats = []
for player in trange(len(mlb_roster)):
    player_position = mlb_roster[player]["position_txt"]
    if player_position == "P":
        time.sleep(.25)
        continue
    else:
        player_id = mlb_roster[player]["player_id"]
        player_name = mlb_roster[player]["name_display_first_last"]
        player_debut = mlb_roster[player]["pro_debut_date"][0:4]

        birth_date_str = mlb_roster[player]["birth_date"][0:10]
        birth_date_dt = datetime.strptime(birth_date_str, "%Y-%m-%d")
        for year in seasons:
            if year == "2022":
                year_stats = {
                    'team_id': mlb_roster[player]["team_id"],
                    'season': '2022',
                    'sport_code': 'mlb',
                    'team_full': mlb_roster[player]["team_name"],
                    'team_abbrev': mlb_roster[player]["team_abbrev"],
                    'player_id': mlb_roster[player]["player_id"],
                    'sport': 'MLB',
                    'player_name': mlb_roster[player]["name_display_first_last"],
                    'player_age': calculate_age(birth_date_dt, year)
                }
                all_stats.append(year_stats)
                continue

            if player_debut == "" or int(year) < int(player_debut):
                break
            else:
                year_stats = api.StatsData().season_hitting_stats(player_id, year)
                if year_stats is None:
                    continue
                if type(year_stats) == list:
                    for i in year_stats:
                        i["player_age"] = calculate_age(birth_date_dt, year)
                        i["player_name"] = player_name
                    all_stats.extend(year_stats)
                else:
                    year_stats["player_age"] = calculate_age(birth_date_dt, year)
                    year_stats["player_name"] = player_name
                    all_stats.append(year_stats)
    time.sleep(.25)

len(all_stats)

# Scrape Yahoo Rankings

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from tqdm import tqdm, trange
import pandas as pd
import datetime
import time


def get_table_data(table_html):
    table_data = []
    for player in table_html:
        try:
            player_data = {}
            player_data["Player Name"] = player.find("div", class_="ysf-player-name").find("a").text.split(" (")[0]
            player_data["Player URL"] = player.find("div", class_="ysf-player-name").find("a").get("href")
            player_data["Team"] = player.find("div", class_="ysf-player-name").find("span").text.split(" - ")[0].upper()
            player_data["Position(s)"] = player.find("div", class_="ysf-player-name").find("span").text.split(" - ")[1]
            player_data["Pre-Season Rank"] = int(player.find("td", class_="Alt Ta-end Nowrap Selected").text)
            player_data["Current Rank"] = int(player.find("td", class_="Ta-end Bdrend").text)
        except:
            continue
        try:
            player_data["Status"] = player.find("span", class_="ysf-player-status").find("abbr").get("title")
        except:
            player_data["Status"] = "OK"

        if len(player_data) > 0:
            table_data.append(player_data)
    return table_data

def setup_driver():
    useragent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f"user-agent={useragent}")
    # chrome_options.add_argument("--log-level=3")
    # chrome_options.headless = True

    CHROME_DRIVER_PATH = os.getenv("CHROME_DRIVER")
    driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH, options=chrome_options)
    return driver

driver = setup_driver()

url = "https://baseball.fantasysports.yahoo.com/b1/51035/players?pspid=782201693&activity=players"
driver.get(url)

yahoo_player_data = []
for i in trange(10):
    soup = bs(driver.page_source, 'html.parser')
    table_data = get_table_data(soup.find_all("tr"))
    yahoo_player_data.extend(table_data)
    sign_in = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[2]/div/div/div[3]/div[1]/div/ul/li[2]')
    sign_in.click()
    time.sleep(1)

df = pd.DataFrame(yahoo_player_data)
df["Player Name"] = df["Player Name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
yahoo_player_data = df.to_dict('records')

In [ ]:
from baseball import api
from tqdm import tqdm, trange
from datetime import datetime, date
import time



def get_mlb_player_data(yahoo_player_data):
    mlb_player_data = []
    for yahoo_player in trange(len(yahoo_player_data)):
        player_name = yahoo_player_data[yahoo_player]["Player Name"]
        try:
            search_result = api.PlayerData().search_for_players(player_name)["search_player_all"]["queryResults"]["row"]
            if type(search_result) == list:
                for j in search_result:
                    i["player_url"] = yahoo_player_data[yahoo_player]["Player URL"]
                    i["yahoo_positions"] = yahoo_player_data[yahoo_player]["Position(s)"]
                    i["yahoo_rank"] = yahoo_player_data[yahoo_player]["Current Rank"]
                    i["yahoo_status"] = yahoo_player_data[yahoo_player]["Status"]
                mlb_player_data.extend(search_result)
            else:
                search_result["player_url"] = yahoo_player_data[yahoo_player]["Player URL"]
                search_result["yahoo_positions"] = yahoo_player_data[yahoo_player]["Position(s)"]
                search_result["yahoo_rank"] = yahoo_player_data[yahoo_player]["Current Rank"]
                search_result["yahoo_status"] = yahoo_player_data[yahoo_player]["Status"]
                mlb_player_data.append(search_result)
        except:
            print(f"Player {player_name} not found.")
        time.sleep(.2)
    return mlb_player_data


def calculate_age(birth_date, season_year):
    today = date(year=int(season_year), month=4, day=1)
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age


def get_mlb_player_stats(mlb_player_data):
    seasons = ["2022", "2021", "2020", "2019", "2018", "2017", "2016"]
    all_stats = []
    for player in trange(len(mlb_player_data)):
        player_position = mlb_player_data[player]["position"]
        if player_position == "P":
            time.sleep(.25)
            continue
        else:
            player_id = mlb_player_data[player]["player_id"]
            player_name = mlb_player_data[player]["name_display_first_last"]
            player_debut = mlb_player_data[player]["pro_debut_date"][0:4]

            birth_date_str = mlb_player_data[player]["birth_date"][0:10]
            birth_date_dt = datetime.strptime(birth_date_str, "%Y-%m-%d")
            for year in seasons:
                if year == "2022":
                    year_stats = {
                        'team_id': mlb_player_data[player]["team_id"],
                        'season': '2022',
                        'sport_code': 'mlb',
                        'team_full': mlb_player_data[player]["team_full"],
                        'team_abbrev': mlb_player_data[player]["team_abbrev"],
                        'player_id': mlb_player_data[player]["player_id"],
                        'sport': 'MLB',
                        'player_name': mlb_player_data[player]["name_display_first_last"],
                        'player_age': calculate_age(birth_date_dt, year),
                        'player_position': player_position
                    }
                    all_stats.append(year_stats)
                    continue

                if player_debut == "" or int(year) < int(player_debut):
                    break
                else:
                    year_stats = api.StatsData().season_hitting_stats(player_id, year)
                    if year_stats is None:
                        continue
                    if type(year_stats) == list:
                        for i in year_stats:
                            i["player_age"] = calculate_age(birth_date_dt, year)
                            i["player_name"] = player_name
                            i["player_position"] = player_position
                        all_stats.extend(year_stats)
                    else:
                        year_stats["player_age"] = calculate_age(birth_date_dt, year)
                        year_stats["player_name"] = player_name
                        all_stats.append(year_stats)
        time.sleep(.2)
    return all_stats

mlb_player_data = get_mlb_player_data(yahoo_player_data)
# mlb_player_stats = get_mlb_player_stats(mlb_player_data)

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver


def setup_driver():
    useragent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f"user-agent={useragent}")
    # chrome_options.add_argument("--log-level=3")
    # chrome_options.headless = True

    CHROME_DRIVER_PATH = os.getenv("CHROME_DRIVER")
    driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH, options=chrome_options)
    return driver

driver = setup_driver()


In [44]:
import pandas as pd

soup = bs(driver.page_source, 'html.parser')
player_table = soup.find("tbody", class_="infinite-scroll-list")
player_list = player_table.find_all("tr")
player_list[0]

<tr class="ys-player Ta-end" data-id="9098"><td class="">133.6</td><td>301</td><td class="Ta-end">C 4</td><td>132.4</td><td class="Ta-start"><span><i class="ys-q-icon-remove Cur-p Icon D-n"></i><span class="CheckBox ys-q-icon-add Pstart-2"><input class="ys-addqueue" data-id="9098" readonly="" type="checkbox"/><svg class="ys-icon-queue CheckBox-Icon-checked Cur(p)" data-icon="star-filled" height="14" style="vertical-align: middle; fill: rgb(0, 0, 0); stroke: rgb(0, 0, 0); stroke-width: 0;" viewbox="0 0 24 24" width="14"><path d="M22.954 8.952c-.126-.39-.49-.652-.9-.652H15.06L12.9 1.652C12.772 1.262 12.41 1 12 1s-.772.263-.898.652L8.94 8.3H1.945c-.41 0-.772.263-.898.652-.127.39.012.815.343 1.056l5.66 4.108-2.163 6.648c-.126.39.012.815.344 1.055.332.24.78.24 1.112 0L12 17.71l5.66 4.11c.165.12.36.18.555.18.194 0 .39-.06.555-.18.33-.24.47-.667.343-1.056l-2.16-6.648 5.658-4.108c.332-.24.47-.667.344-1.056z"></path></svg><svg class="ys-icon-queue CheckBox-Icon Cur(p)" data-icon="star" height=

In [57]:
import pandas as pd

pd.read_html(str(player_table))[0]

,Comp Exp Rank,Rank,Pos Rank,ADP 💎,Player,AB,H,R,HR,RBI,SB,AVG,IP,W,SV,K,ERA,WHIP,Unnamed: 18
0,229.4,573.0,C 21,250.0,Alejandro KirkTor – C,309.0,80.0,40.0,11.0,54.0,0.0,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,234.8,365.0,3B 25,260.3,Jeimer CandelarioDet – 3B,527.0,137.0,69.0,15.0,64.0,1.0,0.260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,240.4,405.0,2B 35,249.7,"Kiké HernándezBos – 2B,OF",462.0,112.0,71.0,18.0,61.0,2.0,0.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,245.0,422.0,3B 33,244.1,Spencer TorkelsonDet – 3BNA,366.0,90.0,62.0,18.0,59.0,2.0,0.246,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,247.0,600.0,C 25,249.1,Elias DíazCol – C,363.0,89.0,46.0,13.0,43.0,0.0,0.245,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,320.5,469.0,SP 140,251.9,Kyle GibsonPhi – SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.0,10.0,0.0,163.0,4.18,1.31,NaN
77,322.0,202.0,RP 46,258.8,Daniel HudsonLAD – RP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,3.0,14.0,70.0,3.50,1.11,NaN
78,322.5,452.0,OF 105,252.3,Jarren DuranBos – OFNA,356.0,92.0,58.0,10.0,34.0,15.0,0.258,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,322.7,364.0,SS 32,-,Miguel RojasMia – SS,471.0,130.0,60.0,8.0,49.0,12.0,0.276,NaN,NaN,NaN,NaN,NaN,NaN,NaN
